In [1]:
import requests
import json
from collections import OrderedDict
from datetime import datetime
from __future__ import print_function 
import mysql.connector 
from mysql.connector import errorcode
import numpy_financial
from tabulate import tabulate
import time

In [2]:
%run ./Functions_Folder/Functions.ipynb
%run ./Functions_Folder/Forecasting_Functions.ipynb

Creating table Company_Info: already exists.
Creating table CalcTable: already exists.
Creating table Yahoo_Forecast: already exists.
Creating table futureEarningsDate: already exists.


In [3]:
cnx = mysql.connector.connect(user='root', password = 'password', database = 'SEC')
cursor = cnx.cursor()

query = ("Select calctable.Ticker as Ticker, calctable.CIK as CIK "
        "From calctable "
        "inner join yahoo_forecast on yahoo_forecast.CIK = calctable.CIK "
        "Where (val > 0 and form = '5YoYAvg' and TableRef = 'Revenue' and frame Like 'TTM%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or (val > 0 and form = '3YoYAvg' and TableRef = 'Revenue' and frame Like 'TTM%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or (val > 0 and form = '1YoYAvg' and TableRef = 'Revenue' and frame Like 'TTM%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or (val > 0 and form = '5YoYAvg' and TableRef = 'earnings_per_share' and frame Like 'CY%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or (val > 0 and form = '3YoYAvg' and TableRef = 'earnings_per_share' and frame Like 'CY%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or (val > 0 and form = '1YoYAvg' and TableRef = 'earnings_per_share' and frame Like 'CY%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or ( 10  > val and val > 0 and form = '5Avg' and TableRef = 'total_ltl_per_lfcf' and frame Like 'CY%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or ( 10  > val and val > 0 and form = '3Avg' and TableRef = 'total_ltl_per_lfcf' and frame Like 'CY%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "or ( 10  > val and val > 0 and form = '1Avg' and TableRef = 'total_ltl_per_lfcf' and frame Like 'CY%' and Rev_gr > 0 and nextYR_gr > 0 and Current_Earnings<Future_Earnings and Current_PE > 0) " 
        "group by calctable.Ticker "
        "having count(calctable.Ticker) >= 5 ")

cursor.execute(query)
List_of_Companies = []
for (Ticker, CIK) in cursor:
    List_of_Companies.append(Ticker)
    
print(len(List_of_Companies))   
# print(List_of_Companies)

651


In [4]:
Updating_Yahoo_Table(List_of_Companies)

In [10]:
error_list = []
counter = 0 
Data = []
Header = ['Ticker', 'NPV', 'Perc w/ 83%', 'Forecast','Price', 'FMV', 'FMV*83%', 'Ending_PE', 'Current_PE', 'Div CAGR', 'Div Yr 1']

List = List_of_Companies


for Ticker in List:
    print
    counter = counter +1 
    try:
        # return (fiveYR_gr, close, Current_Earnings, Future_Earnings, Current_PE, Rev_gr, sic)
        Yahoo_Data = yahoo_forecastTable(Ticker)
        Forecast = Yahoo_Data[0]
        Price = Yahoo_Data[1]
        Current_Earnings = Yahoo_Data[2]
        Furture_Earnings = Yahoo_Data[3]
        Current_PE = Yahoo_Data[4]
        Rev_GR = Yahoo_Data[5]
        
        Forecast = min(Forecast, Rev_GR)
        
        # Note is is a percentage so 15 means the comapanys PE is expected to be reduced by 15%
        if Current_PE>60:
            Decreasing_PE = 50
        elif Current_PE>50:
            Decreasing_PE = 40
        elif Current_PE>40:
            Decreasing_PE = 30
        elif Current_PE>30: 
            Decreasing_PE = 25
        elif Current_PE>15: 
            Decreasing_PE = 10
        else:
            Decreasing_PE = 5
        
        sic = Yahoo_Data[6]
        
        if sic_grouping[sic]['Avg_PE']<0 or sic_grouping[sic]['Avg_PE'] >= Current_PE:
            Ending_PE = Current_PE*(1-(Decreasing_PE/100))
        else:                
            Ending_PE = sic_grouping[sic]['Avg_PE']*.75        
    
        Ending_Earnings = Current_Earnings*((1+(Forecast/100))**5)

        PE_GL = ((Price*Ending_PE)/Current_PE)-Price
        Earning_GL = (Ending_Earnings-Current_Earnings)*Ending_PE
        
        try:
            Div_Data = CAGR(Ticker, 'div_per_share', 3)
            CAGRbySetYears = min(Div_Data[0],.08)
#           Number is always wrong so i /2
            D0 = round(Div_Data[1],2)/2
            # This is a check to see if the Dividend yield is reasonable
            if D0/Price>.10:
                x=1/0
            D1 = round(D0*(1+CAGRbySetYears)**(1),2)
            D2 = round(D0*(1+CAGRbySetYears)**(2),2)
            D3 = round(D0*(1+CAGRbySetYears)**(3),2)
            D4 = round(D0*(1+CAGRbySetYears)**(4),2)
            D5 = round(D0*(1+CAGRbySetYears)**(5),2)
        except:
            D0 = 0 
            D1 = 0 
            D2 = 0 
            D3 = 0 
            D4 = 0 
            D5 = 0 
            CAGRbySetYears = 0 
            
        Yr0 = -Price+D0
        Yr1 = D1
        Yr2 = D2
        Yr3 = D3
        Yr4 = D4
        Yr5 = PE_GL+Earning_GL+Price+D5
        CashFlow = [Yr0, Yr1, Yr2, Yr3, Yr4, Yr5]
        
        
        NPV = round(numpy_financial.npv(.13, CashFlow),2)
        numerator = ((Price+NPV)*.83) - Price
        
        perc = round((numerator/Price)*100,2)
        Row = [Ticker,NPV, perc, Forecast, Price, Price+NPV, round((Price+NPV)*.83,2), round(Ending_PE,2), Current_PE, round(CAGRbySetYears,2), D1]
        Data.append(Row)
    except TypeError:
        # This is for missign data inyahoo forecast
        x=1
    except Exception as err:
        print(err, Ticker)
        error_list.append(Ticker)
        
    perc = round((counter /len(List))*100,2)
    string = """We have {} companies remaining! We are {}% done!                                      """.format(len(List)-counter, perc) 
    print(string, end='\r')

Sort_data = Data
Sort_data.sort(key = lambda i: i[2])
Sort_data.reverse()
Improved_Data = []
Improved_Data.append(Header)
for items in Sort_data:
    Improved_Data.append(items)
print("")
print("")
print(tabulate(Improved_Data, headers='firstrow'))


local variable 'fiveYR_gr' referenced before assignment MOG.A                             
We have 0 companies remaining! We are 100.0% done!                                        

Ticker         NPV    Perc w/ 83%    Forecast    Price      FMV    FMV*83%    Ending_PE    Current_PE    Div CAGR    Div Yr 1
--------  --------  -------------  ----------  -------  -------  ---------  -----------  ------------  ----------  ----------
CELH        298.99         258.8        53       89.98   388.97     322.85       -35.32        -37.18           0           0
ALB         691.13         252.86       52.5    212.57   903.7      750.07         9.37          9.86           0           0
ACMR          9.7           55.21       29.4     11.15    20.85      17.31        13.4          14.11           0           0
HALO         25.43          45.95       29.2     33.53    58.96      48.94        13.65         15.17           0           0
GTLS         76.72          40.95       28.3    109.89   186.

In [9]:
basic_overview('CROX')

                        4yrs        3yrs        2yrs        1yrs        Current       1CAGR    3CAGR  5CAGR
----------------------  ----------  ----------  ----------  ----------  ----------  -------  -------  -------
Revenue                 1.09 bil    1.23 bil    1.39 bil    2.31 bil    3.55 bil      53.67    42.42  28.28
Cash                    213.2 mil   171.97 mil  187.35 mil  142.97 mil  191.63 mil   -10.12    20.97  2.17
Total_Debt              771.39 mil  2.88 bil    2.77 bil    2.62 bil    2.32 bil     201.07   124.59  411.7
Earnings_Per_Share      -1.01       1.66        4.56        11.39       8.71         -23.53    73.77  N/A
Levered_Free_Cash_Flow  91.95 mil   42.55 mil   209.44 mil  458.77 mil  397.25 mil   -13.41   110.56  44.72
Total_Debt_to_Equity    54.78       8.24        5.63        4.15        2.84         -94.82    22.37  N/A
